In [19]:
#all imports used
import requests
import pandas as pd
from tensorflow.keras import Sequential
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.layers import LSTM, Dense
import numpy as np

In [20]:
#thinking ahead of mvp
#optimised function to take in different cryptos
def get_crypto_data(crypto_name):
    
    #sources API coingecko
    url = f"https://api.coingecko.com/api/v3/coins/{crypto_name}/market_chart?vs_currency=usd&days=max&interval=daily"
    response = requests.get(url)
    data = response.json()['prices']
    
    #creates DataFrame with columns date and price
    df = pd.DataFrame(data, columns=['date', 'price'])
    df['date'] = pd.to_datetime(df['date'], unit='ms')
    df.set_index('date', inplace=True)
    df.index = df.index.date

    df = df.groupby(df.index).max()
    return df

df = get_crypto_data("bitcoin")
df

,price
2013-04-28,135.300000
2013-04-29,141.960000
2013-04-30,135.300000
2013-05-01,117.000000
2013-05-02,103.430000
...,...
2023-03-08,22217.209885
2023-03-09,21712.651194
2023-03-10,20376.320007
2023-03-11,20195.228950


In [21]:
# Prepare data
price_data = df['price'].values

#How many previous prices the model will use to predict the target
past_days = 7

In [22]:
X = []
y = []
for i in range(len(price_data) - past_days):
    X.append(price_data[i:i+past_days])
    y.append(price_data[i+past_days])

X = np.array(X)
y = np.array(y)

In [23]:
#Check to see if the shapes line up (5 X's to predict single y)
X_shape_check = pd.DataFrame(X.reshape((X.shape[0], -1)), columns=["dn-1", "dn-2", "dn-3", "dn-4", "dn-5", "dn-6", "dn-7"])
y_shape_check = pd.DataFrame(y, columns=["price"])
X_shape_check.shape, y_shape_check.shape

((3597, 7), (3597, 1))

In [24]:
X_shape_check

,dn-1,dn-2,dn-3,dn-4,dn-5,dn-6,dn-7
0,135.300000,141.960000,135.300000,117.000000,103.430000,91.010000,111.250000
1,141.960000,135.300000,117.000000,103.430000,91.010000,111.250000,116.790000
2,135.300000,117.000000,103.430000,91.010000,111.250000,116.790000,118.330000
3,117.000000,103.430000,91.010000,111.250000,116.790000,118.330000,106.400000
4,103.430000,91.010000,111.250000,116.790000,118.330000,106.400000,112.640000
...,...,...,...,...,...,...,...
3592,23155.543747,23634.333489,23464.336542,22367.433022,22348.329915,22421.885806,22415.113660
3593,23634.333489,23464.336542,22367.433022,22348.329915,22421.885806,22415.113660,22217.209885
3594,23464.336542,22367.433022,22348.329915,22421.885806,22415.113660,22217.209885,21712.651194
3595,22367.433022,22348.329915,22421.885806,22415.113660,22217.209885,21712.651194,20376.320007


In [25]:
y_shape_check

,price
0,116.790000
1,118.330000
2,106.400000
3,112.640000
4,113.000000
...,...
3592,22217.209885
3593,21712.651194
3594,20376.320007
3595,20195.228950


In [26]:
from sklearn.model_selection import train_test_split

#Leave 20% to test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2877, 7), (720, 7), (2877,), (720,))

In [27]:
# Set up LSTM model
model = models.Sequential()
model.add(layers.LSTM(64, activation='relu', input_shape = (past_days, 1)))
model.add(layers.Dense(32, activation = "relu"))
model.add(layers.Dense(1, activation='linear'))

In [28]:
# Compile model
model.compile(loss='mse', optimizer='adam', metrics = ["mae"])

In [29]:
#Model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 64)                16896     
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________


In [30]:
#Fit the model
history = model.fit(X_train, y_train,
                        validation_split = 0.3,
                        shuffle = False,
                        batch_size = 32,
                        epochs = 15,
                        verbose = 0)

In [31]:
# Evaluate performance on test data
loss, mae = model.evaluate(X_test, y_test, verbose=0)
print(loss, mae)

1578743.125 558.07470703125


In [32]:
# Get the last few prices, dependent on past_days
latest_prices = df['price'].values[-past_days:]
latest_prices

array([22421.88580585, 22415.11365969, 22217.20988491, 21712.65119391,
       20376.32000743, 20195.22895027, 21392.78119613])

In [33]:
X = np.append(X, latest_prices)[-past_days:]

In [34]:
# Reshape X to match the input shape of the LSTM model
X = X.reshape((1, past_days, 1))

# Use the trained model to predict the next price
next_price = model.predict(X, verbose = 0)[0][0]
next_price

21750.2